In [111]:
import pandas as pd 

from general_functions import get_file_contents

key = 'zaeHKUd+udWXYAS90iNFVdplnQ58mV0a++SuSZrKRVhjD2cJOhRSDZ9xzMmrxmd5S0TnN8Jg2UZCJLH8Y6oZQg=='
#filename = 'ServiceKey.txt'
#api_key = get_file_contents(filename)

import requests


In [16]:
def load_xml_to_dataframe(response, keyword):
    import bs4 as bs
    import urllib.request
    soup = bs.BeautifulSoup(response_xml,'xml')
    
    rows = soup.find_all(keyword)
    columns = rows[0].find_all()
    
    rowList = []
    nameList = []
    columnList = []

    rowsLen = len(rows)
    columnsLen = len(columns)

    for i in range(0, rowsLen):
        columns = rows[i].find_all()

        for j in range(0, columnsLen):
            if i == 0:
                nameList.append(columns[j].name) #header
            eachColumn = columns[j].text #value
            columnList.append(eachColumn)
        rowList.append(columnList)
        columnList = []    # 다음 row의 값을 넣기 위해 비워준다

    result = pd.DataFrame(rowList, columns=nameList)
    return result

### 계열회사조회


In [154]:
#xml
params ={'serviceKey' : key, 
         'pageNo' : '1', 
         'numOfRows' : '10', 
         'resultType' : 'xml', 
         'basDt' : '20220701' , 
         'crno' : '1101110028939', #계열회사법인등록번호
         'afilCmpyNm' : '케이비아이알로이(주)' }

url = 'http://apis.data.go.kr/1160100/service/GetCorpBasicInfoService/getAffiliate'
response_xml = requests.get(url, params=params).text.encode('utf-8')


import xmltodict
xmltodict.parse(response_xml)

In [155]:
load_xml_to_dataframe(response_xml, 'item') #.to_excel('계열사회사조회.xlsx')

,afilCmpyCrno,afilCmpyNm,basDt,crno,lstgYn
0,1244110014077,케이비아이알로이(주),20220701,1101110028939,


### 기업개요 조회

In [149]:
url = 'http://apis.data.go.kr/1160100/service/GetCorpBasicInfoService/getCorpOutline'
#params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '10', 'resultType' : 'xml', 
#         'basDt' : '20200509', 'crno' : '1101113892240', 'corpNm' : '메리츠자산운용' }


params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '10', 'resultType' : 'xml', 
         'basDt' : '20220701'}
response_xml = requests.get(url, params=params).content #.text.encode('utf-8')

load_xml_to_dataframe(response_xml, 'item').to_excel('기업개요조회.xlsx')

### 연결 대상 종속 기업조회

In [150]:
url = 'http://apis.data.go.kr/1160100/service/GetCorpBasicInfoService/getConsSubsComp'
params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '10', 'resultType' : 'xml', 
         'basDt' : '20200327' }#, 'crno' : '1101110035835', 'sbrdEnpNm' : 'Saudi-Taihan Co. Ltd.' }


response_xml = requests.get(url, params=params).text.encode('utf-8')
load_xml_to_dataframe(response_xml, 'header')

,resultCode,resultMsg
0,00,NORMAL SERVICE.


In [142]:
response_xml


b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<response>\n    <header>\n        <resultCode>00</resultCode>\n        <resultMsg>NORMAL SERVICE.</resultMsg>\n    </header>\n    <body>\n        <numOfRows>10</numOfRows>\n        <pageNo>1</pageNo>\n        <totalCount>0</totalCount>\n        <items/>\n    </body>\n</response>\n'